In [ ]:
#!conda update -y -n base -c defaults conda

In [ ]:
!conda install -y scipy seaborn

In [ ]:
import pandas as pd
import requests
from io import BytesIO
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
s=requests.get(url).content
confirmed = pd.read_csv(BytesIO(s))

In [ ]:
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
s=requests.get(url).content
deaths = pd.read_csv(BytesIO(s))

In [ ]:
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"
s=requests.get(url).content
recovered = pd.read_csv(BytesIO(s))

In [ ]:
OFFSET = 100

In [ ]:
def by_country(df):
    return df.drop(columns=['Province/State', 'Lat', 'Long']).groupby('Country/Region').sum().sort_values([df.columns.tolist()[-1]], ascending=False)

confirmed_country = by_country(confirmed)
deaths_country = by_country(deaths)
recovered_country = by_country(recovered)

In [ ]:
confirmed_country.head(20)

In [ ]:
active_country = confirmed_country.sub(deaths_country).sub(recovered_country).reindex(index=confirmed_country.index)
active_country.head(10)

In [ ]:
pd.DataFrame(confirmed_country.loc['Korea, South']).T

In [ ]:
pd.DataFrame(deaths_country.loc['Germany']).T

In [ ]:
pd.DataFrame(recovered_country.loc['Germany']).T

In [ ]:
def relative(df, offset=OFFSET):
    rows = []
    for country, row in df.iterrows():
        day_n = 0
        data = dict()
        data['country'] = country
        for date, value in row.iteritems():
            if confirmed_country.loc[country][date] >= offset or day_n > 0:
                data[day_n] = value
                day_n += 1
        rows.append(data)
    return pd.DataFrame.from_records(rows, index='country').dropna(how='all')

In [ ]:
def plottable(df, countries=['China', 'Others', 'Cruise Ship'], mode='exclude'):
    filtered = df[df.index.isin(countries)] if mode == 'include' else df[~df.index.isin(countries)]
    plot_data = filtered.head(8).T.dropna(how='all')
    return plot_data

In [ ]:
def plot_linear(plot_data, what='Confirmed', offset=OFFSET):
    fig, ax = plt.subplots(figsize=(10,6), dpi=150)
    sns.lineplot(ax=ax, data=plot_data, dashes=False, markers=True, hue='country')
    plt.legend(loc='upper left')
    ax.set(xlabel=('Day N after the first %d Confirmed Cases' % offset), ylabel=('%s Cases' % what))
    return ax

In [ ]:
def plot_log(plot_data, what='Confirmed', growth_rate=0.30, offset=OFFSET, ymin=80):
    fig, ax = plt.subplots(figsize=(10,5.5), dpi=150)
    sns.lineplot(ax=ax, data=plot_data, dashes=False, markers=True, hue='country')
    plt.legend(loc='upper left')
    ax.set(xlabel=('Day N after the first %d Confirmed Cases' % offset), ylabel=('%s Cases' % what))
    ax.set_yscale('log')
    ax.set_ylim(bottom=ymin)
    x = np.arange(0, len(plot_data), 1)
    y = offset*np.exp(x*growth_rate)

    plt.plot(x, y, 'k-')
    return ax

In [ ]:
plot_linear(plottable(relative(confirmed_country)))

In [ ]:
plot_linear(plottable(relative(active_country)), what='Active')

In [ ]:
plot_log(plottable(relative(confirmed_country)))

In [ ]:
plot_log(plottable(relative(active_country)), what='Active')

In [ ]:
plot_linear(plottable(relative(active_country), countries=['Other']), what='Active')

In [ ]:
plot_log(plottable(relative(active_country), countries=['Other']), what='Active')

In [ ]:
plottable(active_country).index

In [ ]:
plot_data = plottable(active_country, countries=['Other']).iloc[20:]
what = 'Active'
plot_data = plot_data.reset_index().rename(columns={"index":"day"})
fig, ax = plt.subplots(figsize=(10,6), dpi=150)
plot_data.plot.line(ax=ax, style='-', rot=45, xticks=plot_data.index)
plt.legend(loc='upper left')
ax.set(ylabel=('%s Cases' % what))
ax.set_xticklabels(plot_data["day"], fontsize='x-small', horizontalalignment='right');

In [ ]:
plot_data = plottable(deaths_country, countries=['Other']).iloc[30:]
what = 'Deaths'
plot_data = plot_data.reset_index().rename(columns={"index":"day"})
fig, ax = plt.subplots(figsize=(10,6), dpi=150)
plot_data.plot.line(ax=ax, style='.-', rot=45, xticks=plot_data.index)
plt.legend(loc='upper left')
ax.set(ylabel=('%s' % what))
ax.set_xticklabels(plot_data["day"], fontsize='x-small', horizontalalignment='right');

In [ ]:
url="https://services9.arcgis.com/N9p5hsImWXAccRNI/arcgis/rest/services/Z7biAeD8PAkqgmWhxG2A/FeatureServer/1/query?f=json&where=Confirmed%20%3E%200&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=Confirmed%20desc%2CCountry_Region%20asc%2CProvince_State%20asc&resultOffset=0&resultRecordCount=250&cacheHint=true"
s=requests.get(url).json()
d=[feature['attributes'] for feature in s['features']]
df=pd.DataFrame(d)

In [ ]:
active_country.head()

In [ ]:
df1=df[['Country_Region', 'Active', 'Deaths', 'Confirmed', 'Recovered']]
grouped=df1.rename(columns={"Country_Region":"Country/Region"}).groupby('Country/Region').sum().sort_values(['Confirmed'], ascending=False)
grouped.head()

In [ ]:
plot_log(plottable(relative(confirmed_country), countries=['China', 'Others', 'Cruise Ship', 'Iran', 'Korea, South', 'Italy']), growth_rate=0.30, offset=100)

In [ ]:
plot_log(plottable(relative(active_country), countries=['China', 'Others', 'Cruise Ship', 'Iran', 'Korea, South', 'Italy']), growth_rate=0.30, offset=100, what='Active')

In [ ]:
plot_log(plottable(relative(confirmed_country), countries=['China', 'Others', 'Cruise Ship', 'Iran', 'Korea, South']), growth_rate=0.30, offset=100)

In [ ]:
plot_log(plottable(relative(deaths_country)), what='Death', ymin=1, offset=1)

In [ ]:
plot_linear(plottable(relative(confirmed_country, offset=30), countries=['Czechia', 'Germany', 'Italy'], mode='include').head(10), offset=30)
#plot_log(plottable(relative(confirmed_country), countries=['China', 'Others', 'Cruise Ship', 'Iran', 'Korea, South']), growth_rate=0.30, offset=100)